In [2]:
# importing Libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

In [6]:
# Using skimage for image processing
from skimage import data, color, feature
import skimage.data

# HOG: Histogram Of Oriented Gradients
image = color.rgb2gray(data.chelsea())

# Setting visualise=True in feature.hog() returns a visualization of the HOG features, aiding in debugging and improving feature extraction efficacy.
hog_vec, hog_vis = feature.hog(image, visualise=True)

fig, ax = plt.subplots(1, 2, figsize=(12, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
ax[0].imshow(image, cmap='gray')
ax[0].set_title('input image')

ax[1].imshow(hog_vis)
ax[1].set_title('visualization of HOG features');


TypeError: hog() got an unexpected keyword argument 'visualise'